### Input files: Price-Today.csv

In [2]:
import glob
import os
import pandas as pd
from datetime import date, timedelta
from pandas.tseries.offsets import *
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

format_dict = {'qty':'{:,}','volbuy':'{:,}',
              'price_x':'{:,.2f}','price_y':'{:,.2f}','change':'{:,.2f}','price':'{:,.2f}',
              'chg_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}','amount':'{:,.2f}',
              'percent':'{:,.2f}','net':'{:,.2f}','dividend':'{:,.4f}'}

pd.options.display.max_rows = 31
pd.options.display.float_format = '{:.2f}'.format

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2025, 1, 24), datetime.date(2025, 1, 23))

In [3]:
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
yesterday = yesterday.date()
print(f'yesterday: {yesterday}')

today: 2025-01-24
yesterday: 2025-01-23


In [4]:
# If run after actual work day
#today = today - timedelta(days=2)
today

datetime.date(2025, 1, 24)

### Restart and Run All Cells

In [6]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [7]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


## Must rerun whenever there is change in orders

In [9]:
sql = """
SELECT name, date, volbuy, price, active, dividend, period, grade 
FROM buy
ORDER BY name"""
stocks = pd.read_sql(sql, const)
stocks['volbuy'] = stocks['volbuy'].astype('int64')
stocks

,name,date,volbuy,price,active,dividend,period,grade
0,3BBIF,2018-05-17,120000,10.10,1,0.16,3,A3
1,AH,2023-06-08,1200,37.00,1,1.25,1,C1
2,AIMIRT,2023-08-17,12500,10.90,1,0.86,2,A2
3,ASK,2023-02-16,6000,26.25,1,1.16,1,C1
4,ASP,2022-03-07,30000,3.80,1,0.17,1,C1
5,AWC,2023-06-15,9000,4.96,1,0.05,4,B1
6,BCH,2021-09-07,4000,21.70,1,0.37,4,A3
7,CPNREIT,2022-08-16,55000,18.00,1,0.84,2,C1
8,DIF,2020-08-01,35000,13.90,1,0.89,2,B3
9,GVREIT,2022-08-24,60000,7.90,1,0.78,2,C1


In [10]:
names = stocks["name"].values.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'3BBIF', 'AH', 'AIMIRT', 'ASK', 'ASP', 'AWC', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTG', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'TOA', 'TVO', 'WHAIR', 'WHART'"

In [11]:
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [12]:
prices = pd.read_csv(input_file)
prices.shape

(181, 10)

In [13]:
df_merge  = pd.merge(stocks, prices, on = 'name', how='inner')
df_merge

,name,date_x,volbuy,price_x,active,dividend,period,grade,date_y,price_y,change,%change,open,high,low,volume,value(m)
0,3BBIF,2018-05-17,120000,10.10,1,0.16,3,A3,2025-01-24,5.20,0.05,0.97,5.15,5.20,5.10,6887249,35559.35
1,AH,2023-06-08,1200,37.00,1,1.25,1,C1,2025-01-24,13.80,-0.30,-2.13,14.00,14.00,13.70,441483,6113.97
2,AIMIRT,2023-08-17,12500,10.90,1,0.86,2,A2,2025-01-24,10.50,0.10,0.96,10.40,10.50,10.40,145282,1512.79
3,ASK,2023-02-16,6000,26.25,1,1.16,1,C1,2025-01-24,8.85,0.15,1.72,8.75,8.85,8.75,276146,2430.64
4,ASP,2022-03-07,30000,3.80,1,0.17,1,C1,2025-01-24,2.32,0.00,0.00,2.32,2.34,2.32,131322,305.03
5,AWC,2023-06-15,9000,4.96,1,0.05,4,B1,2025-01-24,3.12,0.04,1.30,3.10,3.14,3.08,24676701,76934.20
6,BCH,2021-09-07,4000,21.70,1,0.37,4,A3,2025-01-24,15.00,0.00,0.00,15.00,15.10,14.90,931968,13963.03
7,CPNREIT,2022-08-16,55000,18.00,1,0.84,2,C1,2025-01-24,12.20,0.00,0.00,12.20,12.30,12.10,488639,5978.58
8,DIF,2020-08-01,35000,13.90,1,0.89,2,B3,2025-01-24,8.40,0.00,0.00,8.40,8.45,8.35,2233265,18762.32
9,GVREIT,2022-08-24,60000,7.90,1,0.78,2,C1,2025-01-24,6.55,0.05,0.77,6.50,6.55,6.50,77816,507.23


### Convert column name to make it valid for df.query

In [15]:
pd.options.display.max_rows = 30
df_merge['percent'] = df_merge['%change']
df_merge['chg_amt'] = df_merge['volbuy'] * df_merge['change']
df_merge['volbuy'] = df_merge['volbuy'].astype('int64')
colo = ['name', 'volbuy', 'price_x', 'price_y', 'change', 'percent', 'chg_amt']
filter = df_merge['percent'] != 0
df_pct = df_merge[filter][colo].sort_values(by='percent', ascending=True).copy()
df_amt = df_merge[filter][colo].sort_values(by='chg_amt', ascending=True).copy()

In [16]:
df_pct.shape

(22, 7)

In [17]:
df_amt.shape

(22, 7)

In [18]:
df_pct.query('percent >= 3.00').style.format(format_dict)

,name,volbuy,price_x,price_y,change,percent,chg_amt
22,SINGER,"6,000",24.80,7.10,0.25,3.65,"1,500.00"
20,SCC,600,405.00,156.00,6.00,4.00,"3,600.00"
10,IVL,"7,200",40.00,23.70,1.10,4.87,"7,920.00"
18,PTTGC,"6,000",64.75,22.40,1.70,8.21,"10,200.00"


In [19]:
df_pct.query('percent < -3.00').style.format(format_dict)

,name,volbuy,price_x,price_y,change,percent,chg_amt


In [20]:
df_amt.query('chg_amt >= 6000').style.format(format_dict)

,name,volbuy,price_x,price_y,change,percent,chg_amt
0,3BBIF,"120,000",10.10,5.20,0.05,0.97,"6,000.00"
10,IVL,"7,200",40.00,23.70,1.10,4.87,"7,920.00"
18,PTTGC,"6,000",64.75,22.40,1.70,8.21,"10,200.00"


In [21]:
df_amt.query('chg_amt < -6000').style.format(format_dict)

,name,volbuy,price_x,price_y,change,percent,chg_amt


In [22]:
float_formatter = "฿{:,.2f}"

In [23]:
float_formatter.format(df_amt.query('chg_amt >= 0')['chg_amt'].sum())

'฿55,240.00'

In [24]:
float_formatter.format(df_amt.query('chg_amt < 0')['chg_amt'].sum())

'฿-360.00'

In [25]:
float_formatter.format(df_amt.chg_amt.sum())

'฿54,880.00'